# Creating Binomial Tree

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

## Defining the Instance Variables

When deducing the value of an option (for example) we consider a few key component inherent to the method. These are

- Strike price ($E$)
- Expiration date ($T$)
- Factor for when an increase occurs ($u$)
- Factor for when a decrease occurs ($d$)
- Option type (call or put)
- Contract type (Eurpean or American)

## Defining the Methods For Pricing

The binomial tree method incorporates a delta hedged portfolio

\begin{equation}
\Pi = V - \Delta S
\end{equation}

where

- $\Pi$ is the value of the portfolio ($)
- $V$ is the price of the option ($)
- $\Delta$ is the propotion of stock we should short in order to hedge our portfolio
- $S$ is the stock price ($)


The deifnition of $\Delta$ is $\partial V / \partial S$ but for discrete time cases such as the binomial method $\Delta$ can take a simpler definition. Consider at a single split in the tree the value of the stock price increases to $S^{+}$ ($uS$) such that the underlying increases to $V^-$ or the stock price decreases to $S^{-}$ ($dS$) such that the underlying decreases to $V^-$. Then

\begin{equation}
\Delta = \frac{\partial V}{\partial S} = \frac{V^+ - V^-}{S^+ - S^-} = \frac{V^+ - V^-}{(u-d)S}
\end{equation}

(More information on how this equation is obtained and on the method as a whole is available at
[Investopedia](https://www.investopedia.com/terms/b/binomialoptionpricing.asp).
Alternatively, see books such as *Paul Wilmott Introduced Quantitative Finance* or
*Options, Futures and Other Derivatives* by John C. Hull are great resources.)

In [3]:
def calc_delta(V_p, V_m, u, d, S):
    delta = (V_p - V_m) / (u*S - d*S)
    return delta 